**Description**: save all the stuff sent to the OpenAI API by LangChain as JSONs.

In [1]:
from contextlib import contextmanager
from datetime import datetime
from functools import wraps
import json
import os
from typing import Any, Callable

from langchain_openai import ChatOpenAI

# Use these

Copy these functions somewhere.

Could instead patch the class instead of the object. That way, any
`langchain_openai.ChatOpenAI` object's `create` call will be saved as a JSON. For now,
going to stick to the more conservative instance patch.

In [2]:
@contextmanager
def _monkeypatch_instance_method(obj: Any, method_name: str, new_method: Callable):
    original_method = getattr(obj, method_name)
    # Need to use __get__ when patching instance methods
    # https://stackoverflow.com/a/28127947/18758987
    try:
        setattr(obj, method_name, new_method.__get__(obj, obj.__class__))
        yield
    finally:
        setattr(obj, method_name, original_method.__get__(obj, obj.__class__))


@contextmanager
def _save_method_inputs_as_json(obj: Any, method_name: str, jsons_dir: str):
    # TODO: break if the method takes unnamed args. Use inspect.signature
    original_method = getattr(obj, method_name)

    @wraps(original_method)
    def new_method(self, **kwargs):  # create takes only named args
        current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")
        file_path = os.path.join(jsons_dir, f"{current_time}.json")
        # TODO: consider logging the write
        with open(file_path, mode="w") as file:
            json.dump(kwargs, file, indent=4)
        return original_method(**kwargs)

    with _monkeypatch_instance_method(obj, method_name, new_method):
        yield


@contextmanager
def save_chat_create_inputs_as_jsons(client_with_create_method: Any, jsons_dir: str):
    """
    In this context, save the inputs sent to some API through
    `client_with_create_method.create` in `jsons_dir`.

    Parameters
    ----------
    client_with_create_method : Any
        some object with a `create` method, e.g., `langchain_openai.ChatOpenAI()`. Its
        inputs will be saved as JSONs whenver this method is called
    jsons_dir : str
        directory where your JSONs will get saved. The file names are timestamps

    Example
    -------
    ::

        from langchain_openai import ChatOpenAI

        llm = ChatOpenAI()
        jsons_dir = "temp"

        with save_chat_create_inputs_as_jsons(llm.client, jsons_dir):
            # Note that the llm object is modified in this context, so any code that
            # uses the llm will end up saving a JSON.
            response = llm.invoke("how can langsmith help with testing?")

        # Then look at the json in ./jsons_dir

    Note
    ----
    You probably only need the last JSON that's saved in `jsons_dir` b/c it'll contain
    the chat history.
    """
    with _save_method_inputs_as_json(client_with_create_method, "create", jsons_dir):
        yield

# Minimal demo

In [3]:
llm = ChatOpenAI()

In [4]:
with save_chat_create_inputs_as_jsons(llm.client, jsons_dir="temp"):
    response = llm.invoke("how can langsmith help with testing?")

In [5]:
response

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. Automated testing: Langsmith can be used to write scripts for automated testing of software applications. This can help to ensure that the software functions correctly and meets the specified requirements.\n\n2. Test case generation: Langsmith can be used to generate test cases automatically based on the requirements of the software application. This can help to improve test coverage and ensure that all possible scenarios are tested.\n\n3. Test data generation: Langsmith can be used to generate test data automatically for testing purposes. This can help to ensure that the software application behaves correctly with different types of input data.\n\n4. Test reporting: Langsmith can be used to generate test reports automatically, providing detailed information on the results of the testing process. This can help to identify any issues or defects in the software application.\n\nOverall, Langsmith can help to streaml

# Demo w/ CoT and a tool

The first few cells are just setup. See the last few cells in the notebook.

Demo is from
https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/07-langchain-tools.ipynb

In [6]:
from langchain.tools import BaseTool
from math import pi
from typing import Union


class CircumferenceTool(BaseTool):
    name = "Circumference calculator"
    description = "use this tool when you need to calculate a circumference using the radius of a circle"

    def _run(self, radius: Union[int, float]):
        if isinstance(radius, str):
            # stupid thing. I checked that this happens in the raw demo
            radius = radius.lstrip("radius=: ")
        return float(radius) * 2.0 * pi

In [7]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY") or "OPENAI_API_KEY"

# initialize LLM (we use ChatOpenAI because we'll later define a `chat` agent)
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY, temperature=0, model_name="gpt-3.5-turbo"
)
# initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

In [8]:
from langchain.agents import initialize_agent

tools = [CircumferenceTool()]

# initialize agent with tools
agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method="generate",
    memory=conversational_memory,
)

/Users/kdubey/Envs/fred/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [9]:
jsons_dir = "temp"  # make this on your own
with save_chat_create_inputs_as_jsons(llm.client, jsons_dir):
    agent(
        "I have a circle with diameter 4. First calculate its radius. "
        "That'll be your first observation. "
        "In a new observation, calculate its circumference."
    )

/Users/kdubey/Envs/fred/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
```json
{
    "action": "Circumference calculator",
    "action_input": "radius=2"
}
```
Observation: 12.566370614359172
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The circumference of the circle with a radius of 2 is approximately 12.57 units."
}
```

> Finished chain.


Here's what the last JSON looks like

In [10]:
_json_files = os.listdir(jsons_dir)
_num_jsons = len(_json_files)

last_json_file = sorted(_json_files)[-1]
with open(os.path.join(jsons_dir, last_json_file), "r") as f:
    last_inputs = json.load(f)
print(json.dumps(last_inputs, indent=4))

{
    "messages": [
        {
            "role": "system",
            "content": "Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of 

If the context manager isn't used after it was used, verify that no extra JSONs are
saved, i.e., we're back to using the old `create` method

In [11]:
agent(
    "I have a circle with diameter 4. First calculate its radius. "
    "That'll be your first observation. "
    "In a new observation, calculate its circumference."
)



> Entering new AgentExecutor chain...


```json
{
    "action": "Circumference calculator",
    "action_input": "radius=2"
}
```
Observation: 12.566370614359172
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The circumference of the circle with a radius of 2 is approximately 12.57 units."
}
```

> Finished chain.


{'input': "I have a circle with diameter 4. First calculate its radius. That'll be your first observation. In a new observation, calculate its circumference.",
 'chat_history': [HumanMessage(content="I have a circle with diameter 4. First calculate its radius. That'll be your first observation. In a new observation, calculate its circumference."),
  AIMessage(content='The circumference of the circle with a radius of 2 is approximately 12.57 units.')],
 'output': 'The circumference of the circle with a radius of 2 is approximately 12.57 units.'}

In [12]:
assert len(os.listdir(jsons_dir)) == _num_jsons